In [1]:
import os
import yaml
import time
import random
from pathlib import Path

import pandas as pd
import numpy as np
import torch
import scanpy as sc

import STAGATE_pyG

/rd2/user/xiacr/spatialGLUE/env/stagate/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
adata1 = sc.read_h5ad('./adata1.h5ad')
adata2 = sc.read_h5ad('./adata2.h5ad')
adata1.obs['dataset'] = 'dataset1'
adata2.obs['dataset'] = 'dataset2'

In [11]:
STAGATE_pyG.Cal_Spatial_Net(adata1, k_cutoff=20, model='KNN')
STAGATE_pyG.Cal_Spatial_Net(adata2, k_cutoff=20, model='KNN')
adata = sc.concat([adata1, adata2])
adata.uns['Spatial_Net'] = pd.concat([adata1.uns['Spatial_Net'], adata2.uns['Spatial_Net']])

sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata)

------Calculating spatial graph...
The graph contains 200000 edges, 10000 cells.
20.0000 neighbors per cell on average.
------Calculating spatial graph...
The graph contains 200020 edges, 10001 cells.
20.0000 neighbors per cell on average.


/rd2/user/xiacr/spatialGLUE/env/stagate/conda/lib/python3.8/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [12]:
adata = STAGATE_pyG.train_STAGATE(adata, device='cuda:4')

Size of Input:  (20001, 3957)


100%|██████████| 1000/1000 [01:33<00:00, 10.65it/s]


In [15]:
embd0 = adata[adata.obs['dataset']=='dataset1'].obsm['STAGATE']
embd1 = adata[adata.obs['dataset']=='dataset2'].obsm['STAGATE']

In [16]:
np.savetxt('./stagate_emb0.csv', embd0, delimiter=',')
np.savetxt('./stagate_emb1.csv', embd1, delimiter=',')